In [ ]:
#Include a new person in the client table. His name is Popov Ilya, his email is popov@test and he lives in Moscow.

INSERT INTO client (name_client, city_id, email)
SELECT "Попов Илья", city_id, "popov@test"
FROM city
WHERE name_city = "Москва";

In [ ]:
#Create a new order for Ilya Popov. His comment for the order is 'Contact me about delivery'.

INSERT INTO buy (buy_description, client_id) 
SELECT "Связаться со мной по вопросу доставки", client_id
FROM client
WHERE name_client = "Попов Илья";

In [ ]:
#Add order number 5 to the buy_book table. This order should contain two copies of Pasternak's "Lyrics" and one copy of Bulgakov's "White Guard".

INSERT INTO buy_book (buy_id, book_id, amount)
SELECT 5, book_id, 2 
FROM book
WHERE title = "Лирика";
    
INSERT INTO buy_book (buy_id, book_id, amount)
SELECT 5, book_id, 1 
FROM book
WHERE title = "Белая гвардия";

#Alternative solution
INSERT INTO buy_book (buy_id, book_id, amount)
VALUES (5, (SELECT book_id FROM book WHERE title = "Лирика"), 2);

INSERT INTO buy_book (buy_id, book_id, amount)
VALUES (5, (SELECT book_id FROM book WHERE title = "Белая гвардия"), 1); 

In [ ]:
#Reduce the number of those books in stock that were included in order number 5.

UPDATE book, buy_book
SET book.amount = book.amount - buy_book.amount
WHERE book.book_id = buy_book.book_id AND buy_book.buy_id = 5;

In [ ]:
#Create an invoice (buy_pay table) to pay for the order with number 5, including the name of the books, their author, price, number of books ordered and cost. Name the last column Cost. Enter the information in the table sorted by book title.

CREATE TABLE buy_pay AS 
SELECT title, name_author, price, buy_book.amount, price * buy_book.amount AS Стоимость
FROM 
    book
    INNER JOIN buy_book USING (book_id)
    INNER JOIN author USING (author_id)
WHERE buy_book.buy_id = 5
ORDER BY title;
    
SELECT * FROM buy_pay;

In [ ]:
#Create a total invoice (buy_pay table) to pay for the order with order number 5. Where to include the order number, the number of books in the order (column name Quantity) and its total cost (column name Total). Use ONE query for the solution.

CREATE TABLE buy_pay AS 
SELECT buy_id, sum(Количество) AS Количество, sum(Стоимость) AS Итого
FROM (
    SELECT buy_id, title, name_author, price, buy_book.amount AS Количество, price * buy_book.amount AS Стоимость
    FROM 
        book
        INNER JOIN buy_book USING (book_id)
        INNER JOIN author USING (author_id)
    WHERE buy_book.buy_id = 5
    ORDER BY title) query_1
GROUP BY buy_id
HAVING buy_id = 5;

In [ ]:
#In the buy_step table for order number 5, include all steps from the step table that this order must complete. Enter Null in the date_step_beg and date_step_end columns of all entries.

INSERT INTO buy_step (buy_id, step_id)
SELECT 5, step_id 
    FROM (SELECT DISTINCT(step_id)
        FROM step) q;

SELECT * FROM buy_step;

In [ ]:
#In the buy_step table, enter the date 12.04.2020 for the order number 5.

UPDATE buy_step
SET date_step_beg = "2020.04.12"
WHERE buy_id = 5 AND step_id = 1;

In [ ]:
#Complete the "Payment" step for order number 5 by entering date 13.04.2020 in the date_step_end column and start the next step ("Packing") by entering the same date in the date_step_beg column for this step.

UPDATE buy_step
SET date_step_end= "2020.04.13"
WHERE buy_id = 5 AND step_id = 1;

UPDATE buy_step
SET date_step_beg = "2020.04.13"
WHERE buy_id = 5 AND step_id = 2;